# Core Tutorial

This script will introduce the basics of handling time series data with pynapple.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pynapple as nap
import pandas as pd

## Time series object

Let's create a Tsd object with artificial data. In this example, every time point is 1 second apart. A Tsd object is a wrapper of a [pandas series](https://pandas.pydata.org/docs/reference/series.html).

In [2]:
tsd = nap.Tsd(t = np.arange(100), d = np.random.rand(100), time_units = 's')

print(tsd)

Time (s)
0.0     0.563213
1.0     0.378064
2.0     0.814118
3.0     0.280128
4.0     0.355886
          ...   
95.0    0.172626
96.0    0.984622
97.0    0.972610
98.0    0.748225
99.0    0.678745
Length: 100, dtype: float64


It is possible to toggle between seconds, milliseconds and microseconds. Note that when using *as_units*, the returned object is a simple pandas series.

In [3]:
print(tsd.as_units('ms'))
print(tsd.as_units('us'))

Time (ms)
0.0        0.563213
1000.0     0.378064
2000.0     0.814118
3000.0     0.280128
4000.0     0.355886
             ...   
95000.0    0.172626
96000.0    0.984622
97000.0    0.972610
98000.0    0.748225
99000.0    0.678745
Length: 100, dtype: float64
Time (us)
0           0.563213
1000000     0.378064
2000000     0.814118
3000000     0.280128
4000000     0.355886
              ...   
95000000    0.172626
96000000    0.984622
97000000    0.972610
98000000    0.748225
99000000    0.678745
Length: 100, dtype: float64


Pynapple is able to handle data that only contains timestamps, such as an object containing only spike times. To do so, we construct a Ts object which holds only times. In this case, we generate 10 random spike times between 0 and 100 ms.

In [4]:
ts = nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 'ms')

print(ts)

Time (s)
0.000694   NaN
0.002771   NaN
0.011280   NaN
0.017841   NaN
0.017896   NaN
0.027838   NaN
0.030595   NaN
0.051214   NaN
0.054824   NaN
0.073896   NaN
dtype: float64


If the time series contains multiple columns, we use a TsdFrame.

In [5]:
tsdframe = nap.TsdFrame(t = np.arange(100), 
                        d = np.random.rand(100,3), 
                        time_units = 's', 
                        columns = ['a', 'b', 'c'])

print(tsdframe)

                 a         b         c
Time (s)                              
0.0       0.402090  0.625281  0.008010
1.0       0.458671  0.878846  0.590229
2.0       0.451745  0.019802  0.636421
3.0       0.519470  0.891268  0.732998
4.0       0.450690  0.153512  0.408216
...            ...       ...       ...
95.0      0.600820  0.242669  0.700129
96.0      0.512757  0.927772  0.578880
97.0      0.981968  0.474559  0.648078
98.0      0.117539  0.792938  0.744750
99.0      0.853042  0.745129  0.351100

[100 rows x 3 columns]


## Interval Sets object

The [IntervalSet](https://peyrachelab.github.io/pynapple/core.interval_set/) object stores multiple epochs with a common time unit. It can then be used to restrict time series to this particular set of epochs.

In [6]:
epochs = nap.IntervalSet(start = [0, 10], end = [5, 15], time_units = 's')

new_tsd = tsd.restrict(epochs)

print(epochs)
print('\n')
print(new_tsd)

   start   end
0    0.0   5.0
1   10.0  15.0


Time (s)
0.0     0.563213
1.0     0.378064
2.0     0.814118
3.0     0.280128
4.0     0.355886
5.0     0.583105
10.0    0.778229
11.0    0.843115
12.0    0.251995
13.0    0.868508
14.0    0.952296
15.0    0.277837
dtype: float64


Multiple operations are available for IntervalSet. For example, IntervalSet can be merged. See the full documentation of the class [here](https://peyrachelab.github.io/pynapple/core.interval_set/#pynapple.core.interval_set.IntervalSet.intersect) for a list of all the functions that can be used to manipulate IntervalSets.

In [7]:
epoch1 = nap.IntervalSet(start=[0], end=[10]) # no time units passed. Default is us.
epoch2 = nap.IntervalSet(start=[5,30],end=[20,45])

epoch = epoch1.union(epoch2)
print(epoch1, '\n')
print(epoch2, '\n')
print(epoch)

   start   end
0    0.0  10.0 

   start   end
0    5.0  20.0
1   30.0  45.0 

   start   end
0    0.0  20.0
1   30.0  45.0


## TsGroup

Multiple time series with different time stamps (.i.e. a group of neurons with different spike times from one session) can be grouped with the TsGroup object. The TsGroup behaves like a dictionary but it is also possible to slice with a list of indexes

In [8]:
my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 1000)), time_units = 's'), # here a simple dictionary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 2000)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 3000)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

print(tsgroup, '\n')
print(tsgroup[0], '\n') # dictionary like indexing returns directly the Ts object
print(tsgroup[[0,2]]) # list like indexing

  Index    Freq. (Hz)
-------  ------------
      0            10
      1            20
      2            30 

Time (s)
0.004007    NaN
0.133850    NaN
0.193702    NaN
0.229563    NaN
0.302664    NaN
             ..
99.697703   NaN
99.749609   NaN
99.843855   NaN
99.937553   NaN
99.940693   NaN
Length: 1000, dtype: float64 

  Index    Freq. (Hz)
-------  ------------
      0            10
      2            30


Operations such as restrict can thus be directly applied to the TsGroup as well as other operations.

In [9]:
newtsgroup = tsgroup.restrict(epochs)

count = tsgroup.count(1, epochs, time_units='s') # Here counting the elements within bins of 1 seconds

print(count)

           0   1   2
Time (s)            
0.5       16  19  36
1.5        8  20  35
2.5        9  27  26
3.5        9  18  27
4.5       13  21  24
10.5      15  17  34
11.5      12  23  28
12.5      12  21  24
13.5       6  16  26
14.5       8  24  27


One advantage of grouping time series is that metainformation can be appended directly on an element-wise basis. In this case, we add labels to each Ts object when instantiating the group and after. We can then use this label to split the group. See the [TsGroup](https://peyrachelab.github.io/pynapple/core.ts_group/) documentation for a complete methodology for splitting TsGroup objects.

In [10]:
label1 = pd.Series(index=list(my_ts.keys()), data = [0,1,0])

tsgroup = nap.TsGroup(my_ts, time_units = 's', label1=label1)
tsgroup.set_info(label2=np.array(['a', 'a', 'b']))

print(tsgroup, '\n')

newtsgroup= tsgroup.getby_category('label1')
print(newtsgroup[0], '\n')
print(newtsgroup[1])


  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0            10         0  a
      1            20         1  a
      2            30         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0            10         0  a
      2            30         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      1            20         1  a


## Time support

A key feature of how pynapple manipulates time series is an inherent time support object defined for Ts, Tsd, TsdFrame and TsGroup objects. The time support object is defined as an IntervalSet that provides the time serie with a context. For example, the restrict operation will automatically update the time support object for the new time series. Ideally, the time support object should be defined for all time series when instantiating them. If no time series is given, the time support is inferred from the start and end of the time series. 

In this example, a TsGroup is instantiated with and without a time support. Notice how the frequency of each Ts element is changed when the time support is defined explicitly.

In [11]:
time_support = nap.IntervalSet(start = 0, end = 200, time_units = 's')

my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 20)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 30)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

tsgroup_with_time_support = nap.TsGroup(my_ts, time_support = time_support)

print(tsgroup, '\n')

print(tsgroup_with_time_support, '\n')

print(tsgroup_with_time_support.time_support) # acceding the time support

  Index    Freq. (Hz)
-------  ------------
      0           0.1
      1           0.2
      2           0.3 

  Index    Freq. (Hz)
-------  ------------
      0          0.05
      1          0.1
      2          0.15 

   start    end
0    0.0  200.0
